In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr

In [ ]:
npr.seed(5998821)

bins = 1000
fraction = 0.04
t = np.linspace(0,100,bins)
sigma = 4.0  # high noise is sigma=4, low-noise is sigma=0.5
datafilename = "timeseries.txt"   # add _lownoise for the low-noise version!

A = 8.3
omega = 2.0*np.pi/35.0 
phi = 0.2*np.pi

f = A * np.sin(omega*t + phi)

bool_array = npr.rand(bins) < fraction

f_error = npr.normal(0,sigma,t[bool_array].size)

f_data = f[bool_array] + f_error

t_data = t[bool_array]
#print(bool_array)
#plt.plot(t[bool_array],f[bool_array],'b-')
plt.errorbar(t_data,f_data,yerr=f_error,fmt='bo')

print("A:",A)
print("Omega:", omega)
print("phi:", phi)

np.savetxt(datafilename,np.c_[t_data,f_data,np.abs(f_error)],fmt="%.6e",
           header="# time [s], flux(t) [arb. units], flux_error [arb. units]")

In [ ]:
def est_error(ydata,yguess,error):
    '''
    Takes in the observed data, our current step's estimated y-values 
    for the model, and our guess for the errors in the data.
    
    Returns sum-of-squares error assuming a 2-parameter model
    '''
    return ((ydata-yguess)**2/(2*error**2)).sum()/(ydata.size-3)


def est_model_vals(A,omega,phi,t):
    '''
    Given x-values and our MCMC code's estimates for A and B, this returns 
    estimated y values that we can compare to the actul data .
    '''
    return A * np.sin(omega*t + phi)


In [ ]:
## The actual MCMC code.

# Initial guesses for A and B
# Outcome should be relatively insensitive to these guesses!
A_old = 6.0
Omega_old = 0.2
phi_old = 0.3

# Range of step sizes that we take (linearly in -dA...dA and -dB...dB).
# This has to be relatively small, but if you make it too small you don't get
# Good sampling of the probability distribution function.
dA=0.2
dOmega=0.01
dphi=0.01

# Total number of points we're going to sample (should be at least 10^4)
Npts = 1000000
N_burn = 1000

# Initial model values and 'error' for our starting position
y_old = est_model_vals(A_old,Omega_old,phi_old,t_data)

err_old = est_error(f_data,y_old,f_error)

# These lists keep track of our Markov Chain and errors so we can get our PDF later.
A_guess = []
Omega_guess = []
Phi_guess = []
errors = []

iter_count = 0

for i in range(Npts):
    
    # Guess for our new point (A,B)
    A_new = A_old + npr.uniform(-dA,dA)
    Omega_new = Omega_old + npr.uniform(-dOmega,dOmega)
    phi_new = phi_old + npr.uniform(-dphi,dphi)
    
    # model values based on the new A and B values
    ynew = est_model_vals(A_new,Omega_new,phi_new,t_data)
    
    # sum of squares value comparing data and model given our estimate
    # of the errors in the data.
    err_new = est_error(f_data, ynew, f_error)
    
    # acceptance probability - ratio of likelihoods for old and new data points.
    p_accept = np.exp(-err_new+err_old)
    
    # if R < p_accept, we keep this point.  Otherwise, keep on moving!
    if npr.random() < p_accept:
        A_old = A_new
        Omega_old = Omega_new
        phi_old = phi_new
        
        err_old = err_new
        
        if iter_count > N_burn:
            A_guess.append(A_old)
            Omega_guess.append(Omega_old)
            Phi_guess.append(phi_old)
            errors.append(err_old)

    iter_count += 1



In [ ]:
plt.plot(Phi_guess,Omega_guess,'b.',phi,omega,'co',markersize=10)
#plt.plot(A_guess[-1],Omega_guess[-1],'gs',markersize=10)
#plt.xlabel('A vals')
#plt.ylabel('B vals')
#plt.title('Markov Chain for estimate of (A,B)')
print(len(A_guess))

In [ ]:
from matplotlib.colors import LogNorm

cts,xbin,ybin,img = plt.hist2d(Phi_guess, Omega_guess, bins=60,norm=LogNorm())
plt.plot(phi,omega,'co',markersize=10)
plt.xlabel('phi')
plt.ylabel('omega')

In [ ]:
plt.plot(A_guess,Omega_guess,'b.',A,omega,'co',markersize=10)
#plt.plot(A_guess[-1],Omega_guess[-1],'gs',markersize=10)
#plt.xlabel('A vals')
#plt.ylabel('B vals')
#plt.title('Markov Chain for estimate of (A,B)')
print(len(A_guess))

In [ ]:
cts,xbin,ybin,img = plt.hist2d(A_guess, Omega_guess, bins=60,norm=LogNorm())
plt.plot(A,omega,'co',markersize=10)
plt.xlabel('A')
plt.ylabel('omega')

In [ ]:

plt.plot(t,f,'r-')
plt.errorbar(t_data,f_data,yerr=f_error,fmt='bo')


In [ ]:
blah = plt.hist(A_guess,bins=100,density=True)
plt.xlabel("A")

In [ ]:
blah = plt.hist(Phi_guess,bins=100,density=True)
plt.xlabel("Phi")

In [ ]:
blah = plt.hist(Omega_guess,bins=100,density=True)
plt.xlabel("Omega")